In [ ]:
##Audio Preprocess
#Dimulai dari import data, memberikan overlay suara gangguan yang terdiri atas 4 kondisi, kemudian augmentasi

In [ ]:
import os
import random
import librosa
import numpy as np
import soundfile as sf
from pathlib import Path
from pydub import AudioSegment
from pydub.playback import play


In [ ]:
#Import suara gangguan (overlay sfx)
nature = AudioSegment.from_mp3('dataset/sfx/Nature SFX.mp3')
angin = AudioSegment.from_mp3('dataset/sfx/Angin SFX.mp3')
hujan = AudioSegment.from_mp3('dataset/sfx/Hujan SFX.mp3')
jalanan = AudioSegment.from_mp3('dataset/sfx/Jalanan SFX.mp3')

#gatau automasinya gmn
suaragangguan = [angin,hujan,jalanan,nature]

In [ ]:
def overlay(filepath,outputpath):
    '''Memberi overlay pada audio file dengan 4 skenario tambahan'''
    file = AudioSegment.from_mp3(filepath)
    name = Path(filepath).stem
    panjang_audio = len(file)
    label = ['angin','hujan','jalanan','nature']
    #print("File Imported")
    i = 0

    for gangguan in suaragangguan:
        #print('masuk for loop')
        if panjang_audio >= len(gangguan):
            max = len(gangguan)
            #print('syarat1')
        if panjang_audio < len(gangguan):
            max = len(gangguan)-panjang_audio #Membatasi maksimal randomize agar tidak mengambil part kosong
            #print('syarat2')
        part = random.randrange(0,max) #Mencari random length
        #print('Dapat Rand Range')
        output = file.overlay(gangguan[part:]) #Memberi overlay pada audiofile
        current_label = label[i]
        i += 1
        #print('Conver Berhasil')
        output.export("{}{}_{}.wav".format(outputpath,name,current_label), format="wav") #Eksport dan memberi label sesuai gangguan
        print("[Overlaying \"{}\" with \"{}\" Succeed]".format(name,current_label))


In [ ]:
def pitch(filepath,semitones,outputpath):
    '''Augmentasi ptich shifting pada data: pitch(path data, semitones,path output data)'''
    name = Path(filepath).stem
    label = ['pitchup','pitchdown']
    file, sr = librosa.load(filepath)
    
    pitchup  = librosa.effects.pitch_shift(file, sr, semitones)
    sf.write("{}{} [{}].wav".format(outputpath,name,label[0]),pitchup,sr)
    pitchdown  = librosa.effects.pitch_shift(file, sr, -semitones)
    sf.write("{}{} [{}].wav".format(outputpath,name,label[1]),pitchdown,sr)
    return print("[Pitch Shifting to \"{}\" Succeed]".format(name))


In [ ]:
def whitenoise(filepath,outputpath):
    '''Augmentasi white noise pada data: whitenoise(path data, path output data)'''
    name = Path(filepath).stem
    file, sr = librosa.load(filepath)
    noise = np.random.normal(0,file.std(), file.size)
    noise_added = file + noise
    sf.write("{}{} [noise].wav".format(outputpath,name),noise_added,sr)
    return print("[Add White Noise to \"{}\" Succeed]".format(name))

In [ ]:
def fadeinout(filepath,outputpath):
    '''Augmentasi fade in atau fade out pada data: fadeinout(path data, path output data)'''
    name = Path(filepath).stem
    label = ['fade out','fade in']
    file, sr = librosa.load(filepath)
    
    end = file.shape[0]
    fadepoint = sr
    #print(fadepoint)
    length = int(fadepoint*sr)    
    #print(length)
    #print('file sudah masuk')
    startfp = end-length

    fadeout_curve = np.linspace(1,0,length)
    fadein_curve = np.linspace(0,1,length)
    #print(fadeout_curve.shape)
    #print(file[-length:].shape)

    fo = file[startfp:]*fadeout_curve
    #print("masuk proses")
    sf.write("{}{} [{}].wav".format(outputpath,name,label[0]),fo,sr)
    print("[Fade Out Applied to \"{}\"]".format(name))
    
    fi = file[startfp:]*fadein_curve
    sf.write("{}{} [{}].wav".format(outputpath,name,label[1]),fi,sr)
    print("[Fade In Applied to \"{}\"]".format(name))

In [ ]:
horn_audio = 'dataset/klakson/'
crash_audio = 'dataset/crash/'
horn_output = 'dataset/edited/klakson/'
horn_output_augment = 'dataset/edited/klakson/pitchshift/'
crash_output = 'dataset/edited/crash/'
crash_output_augment = 'dataset/edited/crash/augment/'
listdir = [horn_audio,crash_audio,horn_output,crash_output]

In [ ]:
for file in sorted(os.listdir(crash_audio)):
    filename = os.fsdecode(file)
    if filename.endswith(".mp3") or filename.endswith(".wav"):
        filepath = os.path.join(crash_audio,filename)
        print("processing {}".format(filepath))
        pitch(filepath,0.5,crash_output_augment)
        fadeinout(filepath,crash_output_augment)
        whitenoise(filepath,crash_output_augment)

In [ ]:
for file in sorted(os.listdir(horn_audio)):
    filename = os.fsdecode(file)
    if filename.endswith(".mp3") or filename.endswith(".wav"):
        filepath = os.path.join(horn_audio,filename)
        pitch(filepath,0.5,horn_output_augment)